In [4]:
from DatabaseConnections import *
import params


STARTING...



In [6]:
db = PostgresDb(params.DB_HOST, params.DB_NAME) 

User name (RIS_TRAINING):shostetter
Password (RIS_TRAINING):········


In [7]:
def reset(db, version):
    tables = ['node_stnameft', 'lion', 'node', 'c_master_node',
              'c_master_segs', 'C_Lion_%s' % version, 'C_Lion_Nodes_%s' % version, 'm']
    for t in tables:
        print 'Droping %s\n' % t
        db.query("DROP TABLE if exists %s" % t)
    return raw_input('Re-imported lion and nodes? <enter>\n').upper()

In [11]:
reset(db, params.VERSION)

Droping node_stnameft

Update sucessfull
Droping lion

Update sucessfull
Droping node

Update sucessfull
Droping c_master_node

Update sucessfull
Droping c_master_segs

Update sucessfull
Droping C_Lion_16d

Update sucessfull
Droping C_Lion_Nodes_16d

Update sucessfull
Droping m

Update sucessfull
Re-imported lion and nodes? <enter>



''

# Preprocess 
## Set up data schema

In [14]:
@timeDec
def pre_process(db, lion, node, version, highways=False):
    print 'Adding version number (%s)...\n' % version
    db.query("alter table %s add version varchar(50)" % lion)
    db.query("alter table %s add version varchar(50)" % node)
    db.query("update %s set version  = '%s'" % (lion, version))
    db.query("update %s set version  = '%s'" % (node, version))
    
    print 'Adding master ID fields to tables...\n'
    db.query("alter table %s add column mft int, add column masteridfrom int, add column masteridto int" % lion)
    db.query("alter table %s add column masterid int" % node)
    db.query("alter table %s add column is_int int" % node)
    
    print 'Creating node street name table (highways = %s)...\n' % str(highways)
    if highways:
        db.query("""drop table if exists node_stnameFT;
                    create table node_stnameFT as (
                    select node::int, street, 0 as master from (
                        select nodeidfrom as node, street
                        from {0}
                            where rb_layer in ('B', 'G') and featuretyp in ('0', '6', 'C') and
                            (nonped = 'D' or nonped is null)
                            and street != 'UNNAMED STREET' and street not like '%{1}%'
                        group by nodeidfrom, street

                        union

                        select nodeidto as node, street
                        from {0}
                        where rb_layer in ('B', 'G') and featuretyp in ('0', '6', 'C') and (nonped = 'D'
                        or nonped is null)
                        and street != 'UNNAMED STREET' and street not like '%{1}%'
                        group by nodeidto, street

                        union
                        /* ---------- ADD BACK IN PELHAM PKWY MAIN LINE ---------- */
                        select nodeidfrom as node, street
                        from {0}
                        where rb_layer in ('B', 'G') and featuretyp in ('0', '6', 'C') and
                        (nonped = 'V' and street = 'PELHAM PARKWAY')
                        and street != 'UNNAMED STREET' and street not like '%{1}%'
                        group by nodeidfrom, street
                        /* ---------- ADD BACK IN PELHAM PKWY MAIN LINE ---------- */
                        union
                        /* ---------- ADD BACK IN FORDHAM RD MAIN LINE ---------- */
                        select nodeidfrom as node, street
                        from {0}
                        where nonped = 'V' and rb_layer in ('B', 'G') and street = 'EAST FORDHAM ROAD'
                        group by nodeidfrom, street
                        /* ---------- ADD BACK IN FORDHAM RD MAIN LINE ---------- */
                        ) ft
                    group by node, street)
                """.format(lion, 'PED OVPS'))
    else:
        db.query("""drop table if exists node_stnameFT;
                    create table node_stnameFT as (
                    select node::int, street, 0 as master
                    from (
                        select nodeidfrom as node, street
                        from {0}
                        where rb_layer in ('B', 'G') and featuretyp in ('0', '6', 'C')
                        and street != 'UNNAMED STREET' and street not like '%{1}%'
                        group by nodeidfrom, street

                        union

                        select nodeidto as node, street
                        from {0}
                        where rb_layer in ('B', 'G') and featuretyp in ('0', '6', 'C')
                        and street != 'UNNAMED STREET' and street not like '%{1}%'
                        group by nodeidto, street
                        ) ft
                    group by node, street)
                """.format(lion, 'PED OVPS'))


In [ ]:
pre_process(db, params.LION, params.NODE, params.VERSION, params.HIGHWAYS)

Adding version number (16d)...

Query Failed:

	alter table lion add version varchar(50)
Query Failed:

	alter table node add version varchar(50)
Update sucessfull
